In [132]:
import pyforest
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder ,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import flaml
from catboost import CatBoostRegressor
import mlflow
from category_encoders import TargetEncoder
from autofeat import AutoFeatRegressor
import math
from flaml import AutoML

In [133]:
df  = pd.read_csv('/Users/karanam.rama/Downloads/house_price_pred_repo/datas/raw/house_pred_raw_data.csv')
df['MSSubClass'] =df['MSSubClass'].astype('str')
# df['price_per_sqft'] = df['SalePrice']/df['LotArea']
## removing the houses with prices greater than 500000
# df = df[df['SalePrice']<500000]
print('shape of the dataframe',df.shape)
print('data frame head')
df.head()

<IPython.core.display.Javascript object>

shape of the dataframe (1460, 81)
data frame head


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [134]:
df['Fence']=df['Fence'].fillna('NA')
df['PoolQC'] = df['PoolQC'].fillna('NA')
df['MiscFeature'] = df['MiscFeature'].fillna('NA')
df['Alley'] = df['Alley'].fillna('NA')
df['Fence'] = df['Fence'].fillna('NA')
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')
df['LotFrontage'] = df['LotFrontage'].fillna(0)

ndf = pd.DataFrame(df.isnull().mean())
ndf.columns = ['null_mean']
ndf = ndf[ndf['null_mean']>0]
ndf['column_name'] = ndf.index


num_list = df[ndf['column_name']].select_dtypes(include=['number']).columns
obj_list = df[ndf['column_name']].select_dtypes(exclude=['number']).columns

# replacing null of int type columns with their mean values
df[num_list] = SimpleImputer(strategy='mean').fit_transform(df[num_list])

# replacing null of object type columns with the most frequent category
df[obj_list] = SimpleImputer(strategy='most_frequent').fit_transform(df[obj_list])

df.set_index(['Id'],inplace = True)

x = df.drop('SalePrice',axis = 1)
y = df['SalePrice']
all_columns = x.columns
cat_columns = x.select_dtypes(include=['object']).columns
num_columns = x.select_dtypes(exclude=['object']).columns

# creating a new data frame with numerical columns only
num_x = x[num_columns]
num_x['Age_of_house'] = num_x['YrSold'] - num_x['YearBuilt']
num_x['years_since_remodelled'] = num_x['YrSold'] - num_x['YearRemodAdd']


# preproccessing the numerical data of x using standard scaler and minmax scaler
ss = StandardScaler()
mms = MinMaxScaler()
num_x_ss = pd.DataFrame(ss.fit_transform(num_x),columns=num_x.columns,index = num_x.index)
num_x_ss_mms  =  pd.DataFrame(mms.fit_transform(num_x_ss),columns=num_x.columns,index = num_x.index)

#preproccessing the categorical data of x using Target Encoder ,standard scaler and minmax scaler
cat_x =  TargetEncoder().fit_transform(x[cat_columns],y)
cat_x_ss = pd.DataFrame(ss.fit_transform(cat_x),columns=cat_x.columns,index = cat_x.index)
cat_x_ss_mms  =  pd.DataFrame(mms.fit_transform(cat_x_ss),columns=cat_x.columns,index = cat_x.index)


# joining the both dataframes 

final_x = num_x_ss_mms.join(cat_x_ss_mms,how='inner')

<IPython.core.display.Javascript object>

<ipython-input-134-4bdf94454cd3>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-134-4bdf94454cd3>:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [127]:
final_x.shape
# 1460

(1460, 81)

In [135]:
x_train,x_test,y_train,y_test = train_test_split(final_x,y,random_state=42,test_size=0.3)

In [116]:
# afr = AutoFeatRegressor(feateng_steps=2)
# x_train_af =  afr.fit_transform(x_train,y_train)
# x_test_af = afr.transform(x_test)
# y_pred = afr.predict(x_test_af)
# rmse = mean_squared_error(y_test,y_pred,squared=False)
# mse = mean_squared_error(y_test,y_pred,squared=True)
# mae = mean_absolute_error(y_test,y_pred)
# explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
# r2_score = sklearn.metrics.r2_score(y_test,y_pred)
# print('rmse ', rmse)
# print('mse ', mse)
# print('mae ', mae)
# print('explained_variance_score',explained_variance_score)
# print('r2_score',r2_score)

In [139]:
lr = LinearRegression()
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2_score = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2_score',r2_score)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  30309.32410818455
mse  918655127.8949772
mae  19558.456621004567
explained_variance_score 0.868883932168989
r2_score 0.8683515642857571


In [140]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(y_test),y=np.array(y_pred),mode ='markers'))
fig.add_shape(type='line',x0 = 0,x1 =755000,y0 =0,y1=755000,line=dict(
        color="Red",
        width=3,
        dash="dashdot",
    ))
fig.update_layout(xaxis_title='true_value',yaxis_title='pred_value')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
xgbr = XGBRegressor()
xgbr.fit(x_train,y_train)
y_pred = xgbr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  25532.854189660542
mse  651926643.0704659
mae  16343.458181720891
explained_variance_score 0.9066353069966839
r2 0.9065752531558545


In [142]:
y_pred = xgbr.predict(x_train)
rmse = mean_squared_error(y_train,y_pred,squared=False)
mse = mean_squared_error(y_train,y_pred,squared=True)
mae = mean_absolute_error(y_train,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_train,y_pred)
r2 = sklearn.metrics.r2_score(y_train,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  758.5730446754746
mse  575433.0641082196
mae  515.11627400318
explained_variance_score 0.9999043906000309
r2 0.999904390487886


In [131]:
lgbmr = LGBMRegressor()
lgbmr.fit(x_train,y_train)
y_pred = lgbmr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  25908.356047612415
mse  671242913.0898547
mae  15956.974665987102
explained_variance_score 0.9038311088329799
r2 0.9038071232508775


In [121]:
cbr = CatBoostRegressor()
cbr.fit(x_train,y_train)
y_pred = lgbmr.predict(x_test)

Learning rate set to 0.039134
0:	learn: 71154.9435271	total: 3.28ms	remaining: 3.28s
1:	learn: 69485.7651474	total: 4.98ms	remaining: 2.49s
2:	learn: 67795.9928357	total: 6.59ms	remaining: 2.19s
3:	learn: 66126.4331148	total: 8.32ms	remaining: 2.07s
4:	learn: 64541.9810199	total: 10ms	remaining: 2s
5:	learn: 63023.3598736	total: 11.8ms	remaining: 1.96s
6:	learn: 61393.6403892	total: 13.4ms	remaining: 1.91s
7:	learn: 59967.2555425	total: 14.9ms	remaining: 1.84s
8:	learn: 58460.1400731	total: 16.7ms	remaining: 1.83s
9:	learn: 57151.7866462	total: 18.1ms	remaining: 1.79s
10:	learn: 55873.7708506	total: 19.5ms	remaining: 1.75s
11:	learn: 54680.6873399	total: 20.9ms	remaining: 1.72s
12:	learn: 53477.4128491	total: 22.4ms	remaining: 1.7s
13:	learn: 52314.6039224	total: 24ms	remaining: 1.69s
14:	learn: 51225.1206289	total: 25.5ms	remaining: 1.68s
15:	learn: 50049.3425057	total: 27.2ms	remaining: 1.67s
16:	learn: 48883.8258991	total: 28.8ms	remaining: 1.67s
17:	learn: 47886.6094645	total: 30.4

239:	learn: 14297.1511869	total: 358ms	remaining: 1.13s
240:	learn: 14261.8387386	total: 360ms	remaining: 1.13s
241:	learn: 14237.9958086	total: 362ms	remaining: 1.13s
242:	learn: 14232.9555280	total: 364ms	remaining: 1.13s
243:	learn: 14227.6451711	total: 366ms	remaining: 1.14s
244:	learn: 14216.3108897	total: 368ms	remaining: 1.14s
245:	learn: 14188.5013795	total: 371ms	remaining: 1.14s
246:	learn: 14175.0461722	total: 373ms	remaining: 1.14s
247:	learn: 14170.5832271	total: 375ms	remaining: 1.14s
248:	learn: 14155.8963765	total: 377ms	remaining: 1.14s
249:	learn: 14121.0031478	total: 379ms	remaining: 1.14s
250:	learn: 14078.6918996	total: 381ms	remaining: 1.14s
251:	learn: 14047.2138918	total: 382ms	remaining: 1.13s
252:	learn: 14034.2411582	total: 384ms	remaining: 1.13s
253:	learn: 14029.6934209	total: 385ms	remaining: 1.13s
254:	learn: 14000.1641992	total: 387ms	remaining: 1.13s
255:	learn: 13969.4286330	total: 388ms	remaining: 1.13s
256:	learn: 13943.1293762	total: 390ms	remaining

500:	learn: 9446.1801813	total: 718ms	remaining: 715ms
501:	learn: 9444.5013278	total: 719ms	remaining: 714ms
502:	learn: 9424.0978910	total: 721ms	remaining: 712ms
503:	learn: 9411.1888804	total: 722ms	remaining: 711ms
504:	learn: 9393.2379207	total: 724ms	remaining: 709ms
505:	learn: 9378.0989082	total: 725ms	remaining: 708ms
506:	learn: 9376.5482594	total: 727ms	remaining: 706ms
507:	learn: 9354.2045885	total: 728ms	remaining: 705ms
508:	learn: 9343.8978090	total: 729ms	remaining: 703ms
509:	learn: 9321.1948051	total: 731ms	remaining: 702ms
510:	learn: 9309.2007340	total: 732ms	remaining: 701ms
511:	learn: 9300.9009071	total: 733ms	remaining: 699ms
512:	learn: 9293.9524692	total: 735ms	remaining: 698ms
513:	learn: 9277.6405333	total: 736ms	remaining: 696ms
514:	learn: 9267.3899708	total: 738ms	remaining: 695ms
515:	learn: 9249.9140652	total: 739ms	remaining: 693ms
516:	learn: 9228.6834221	total: 741ms	remaining: 692ms
517:	learn: 9218.4629812	total: 742ms	remaining: 691ms
518:	learn

765:	learn: 6685.2938580	total: 1.08s	remaining: 330ms
766:	learn: 6684.4907807	total: 1.08s	remaining: 329ms
767:	learn: 6682.4097160	total: 1.08s	remaining: 328ms
768:	learn: 6674.3612760	total: 1.09s	remaining: 326ms
769:	learn: 6673.2841817	total: 1.09s	remaining: 325ms
770:	learn: 6672.3962219	total: 1.09s	remaining: 323ms
771:	learn: 6671.2770722	total: 1.09s	remaining: 322ms
772:	learn: 6663.5027765	total: 1.09s	remaining: 320ms
773:	learn: 6653.5479387	total: 1.09s	remaining: 319ms
774:	learn: 6645.0745361	total: 1.09s	remaining: 318ms
775:	learn: 6637.0216951	total: 1.09s	remaining: 316ms
776:	learn: 6629.8428684	total: 1.1s	remaining: 315ms
777:	learn: 6619.6494563	total: 1.1s	remaining: 313ms
778:	learn: 6614.0687497	total: 1.1s	remaining: 312ms
779:	learn: 6604.0880358	total: 1.1s	remaining: 311ms
780:	learn: 6591.7313826	total: 1.1s	remaining: 309ms
781:	learn: 6579.0948456	total: 1.1s	remaining: 308ms
782:	learn: 6573.1152569	total: 1.11s	remaining: 307ms
783:	learn: 6565

In [122]:
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  23102.196308488743
mse  533711474.2759509
mae  15731.631174056105
explained_variance_score 0.8885252221106991
r2 0.887747151843336


In [143]:
automl = AutoML()
automl_settings = {"time_budget" : 200,"metric" :  'rmse', "task":'regression',"eval_method" : 'cv'}
automl.fit(x_train,y_train,**automl_settings) 

[flaml.automl: 08-20 10:41:56] {1102} INFO - Evaluation method: cv
[flaml.automl: 08-20 10:41:56] {630} INFO - Using RepeatedKFold
[flaml.automl: 08-20 10:41:56] {1123} INFO - Minimizing error metric: rmse
[flaml.automl: 08-20 10:41:56] {1143} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree']
[flaml.automl: 08-20 10:41:56] {1210} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.2s,	best lgbm's error=62486.8289,	best lgbm's error=62486.8289
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 1, current learner lgbm
[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.3s,	best lgbm's error=62486.8289,	best lgbm's error=62486.8289
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 2, current learner lgbm


Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations


[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.4s,	best lgbm's error=47016.3773,	best lgbm's error=47016.3773
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 3, current learner lgbm
[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.6s,	best lgbm's error=41874.8206,	best lgbm's error=41874.8206
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 4, current learner lgbm


Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations


[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.7s,	best lgbm's error=41874.8206,	best lgbm's error=41874.8206
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 5, current learner lgbm
[flaml.automl: 08-20 10:41:57] {1361} INFO -  at 0.8s,	best lgbm's error=38618.3551,	best lgbm's error=38618.3551
[flaml.automl: 08-20 10:41:57] {1210} INFO - iteration 6, current learner lgbm


...

[flaml.automl: 08-20 10:44:45] {1361} INFO -  at 168.3s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:45] {1210} INFO - iteration 253, current learner xgboost
[flaml.automl: 08-20 10:44:50] {1361} INFO -  at 173.3s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:50] {1210} INFO - iteration 254, current learner xgboost
[flaml.automl: 08-20 10:44:53] {1361} INFO -  at 176.8s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:53] {1210} INFO - iteration 255, current learner lgbm
[flaml.automl: 08-20 10:44:55] {1361} INFO -  at 178.8s,	best lgbm's error=26647.7952,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:55] {1210} INFO - iteration 256, current learner extra_tree


Finished loading model, total used 492 iterations


[flaml.automl: 08-20 10:44:56] {1361} INFO -  at 179.1s,	best extra_tree's error=29449.3239,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:56] {1210} INFO - iteration 257, current learner lgbm
[flaml.automl: 08-20 10:44:57] {1361} INFO -  at 180.4s,	best lgbm's error=26647.7952,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:57] {1210} INFO - iteration 258, current learner extra_tree


Finished loading model, total used 226 iterations


[flaml.automl: 08-20 10:44:57] {1361} INFO -  at 180.8s,	best extra_tree's error=29449.3239,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:44:57] {1210} INFO - iteration 259, current learner xgboost
[flaml.automl: 08-20 10:45:03] {1361} INFO -  at 186.6s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:45:03] {1210} INFO - iteration 260, current learner xgboost
[flaml.automl: 08-20 10:45:09] {1361} INFO -  at 192.7s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:45:09] {1210} INFO - iteration 261, current learner xgboost
[flaml.automl: 08-20 10:45:15] {1361} INFO -  at 198.6s,	best xgboost's error=26225.7760,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:45:15] {1210} INFO - iteration 262, current learner extra_tree
[flaml.automl: 08-20 10:45:16] {1361} INFO -  at 199.4s,	best extra_tree's error=29145.9217,	best xgboost's error=26225.7760
[flaml.automl: 08-20 10:45:16] {1407} INFO - 

In [144]:
y_pred = automl.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  23651.94992296881
mse  559414735.1586244
mae  14516.623742508562
explained_variance_score 0.9198886054499145
r2 0.9198327287761575


In [145]:
y_pred = automl.predict(x_train)
rmse = mean_squared_error(y_train,y_pred,squared=False)
mse = mean_squared_error(y_train,y_pred,squared=True)
mae = mean_absolute_error(y_train,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_train,y_pred)
r2 = sklearn.metrics.r2_score(y_train,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  5342.750383424786
mse  28544981.659585703
mae  4324.004036203522
explained_variance_score 0.9952571864265594
r2 0.9952571863870818


In [94]:
automl.best_config

{'n_estimators': 241,
 'max_leaves': 24,
 'min_child_weight': 0.7130293574356446,
 'learning_rate': 0.041688643084278026,
 'subsample': 0.7025417777212305,
 'colsample_bylevel': 0.7981550119871405,
 'colsample_bytree': 0.4129632882571486,
 'reg_alpha': 0.33064460645442645,
 'reg_lambda': 0.017357546739781796}

In [95]:
automl.model.estimator

In [98]:
cbr = XGBRegressor(**automl.best_config)
cbr.fit(x_train,y_train)
y_pred = cbr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  23246.83232886318
mse  540415213.3262783
mae  14572.812883490296
explained_variance_score 0.9225642819008444
r2 0.9225554668882044


In [99]:
ser = pd.DataFrame(cbr.feature_importances_)
ser.columns  =['feature_importance']
ser['column_name'] = x_test.columns
ser = ser.sort_values('feature_importance',ascending=False)
ser.head(20)

<IPython.core.display.Javascript object>

,feature_importance,column_name
56,0.229091,ExterQual
24,0.092835,GarageCars
68,0.073603,KitchenQual
59,0.072160,BsmtQual
2,0.062185,OverallQual
46,0.050656,Neighborhood
17,0.028256,FullBath
14,0.025375,GrLivArea
70,0.024064,FireplaceQu
12,0.019423,2ndFlrSF


In [100]:
def best_features(nl,col_count):
    x_train_new = x_train[nl]
    x_test_new = x_test[nl]
    cbr = XGBRegressor(**automl.best_config)
    cbr.fit(x_train_new,y_train)
    y_pred = cbr.predict(x_test_new)
    rmse = mean_squared_error(y_test,y_pred,squared=False)
    mse = mean_squared_error(y_test,y_pred,squared=True)
    mae = mean_absolute_error(y_test,y_pred)
    explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
    r2 = sklearn.metrics.r2_score(y_test,y_pred)
    templist = []
    templist.append(col_count)
    templist.extend([rmse,mse,mae,explained_variance_score,r2])
    return templist

In [42]:
x_train.shape[1]

82

In [101]:
analysis_list = []
for i in range(1,x_train.shape[1],1):
    nl = list(ser.head(i)['column_name'])
    analysis_list.append(best_features(nl,i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [102]:
an_df = pd.DataFrame(analysis_list)
an_df.columns = ['no_of_features','rmse','mse','mae','explained_variance_score','r2']
an_df.head()

<IPython.core.display.Javascript object>

,no_of_features,rmse,mse,mae,explained_variance_score,r2
0,1,56336.471240,3.173798e+09,39214.366260,0.546114,0.545177
1,2,48129.052146,2.316406e+09,33059.120594,0.668052,0.668046
2,3,46224.947170,2.136746e+09,31739.386273,0.693914,0.693792
3,4,45504.554636,2.070664e+09,31194.911993,0.703420,0.703262
4,5,41037.784404,1.684100e+09,27837.790008,0.758729,0.758659


In [103]:
an_df = an_df[an_df['r2']>0.9]

In [104]:
px.scatter(an_df,x='no_of_features',y='r2')

<IPython.core.display.Javascript object>

In [105]:
list(ser.head(68)['column_name'])

['ExterQual',
 'GarageCars',
 'KitchenQual',
 'BsmtQual',
 'OverallQual',
 'Neighborhood',
 'FullBath',
 'GrLivArea',
 'FireplaceQu',
 '2ndFlrSF',
 'GarageArea',
 'TotRmsAbvGrd',
 'LowQualFinSF',
 'TotalBsmtSF',
 'PoolArea',
 'MSSubClass',
 '1stFlrSF',
 'BldgType',
 'Fireplaces',
 'CentralAir',
 'GarageFinish',
 'MasVnrType',
 'LandSlope',
 'BsmtFinSF1',
 'ScreenPorch',
 'BedroomAbvGr',
 'BsmtFullBath',
 'BsmtFinType1',
 'LandContour',
 'YearBuilt',
 'HalfBath',
 'PoolQC',
 'BsmtExposure',
 'Age_of_house',
 'LotArea',
 'GarageQual',
 'years_since_remodelled',
 'LotShape',
 'Foundation',
 'Condition1',
 'GarageType',
 'KitchenAbvGr',
 'Exterior1st',
 'SaleType',
 'MSZoning',
 'HouseStyle',
 'GarageYrBlt',
 'WoodDeckSF',
 'YearRemodAdd',
 'OpenPorchSF',
 'ExterCond',
 'Exterior2nd',
 'RoofStyle',
 '3SsnPorch',
 'MasVnrArea',
 'BsmtUnfSF',
 'OverallCond',
 'MiscVal',
 'Functional',
 'SaleCondition',
 'BsmtCond',
 'EnclosedPorch',
 'LotFrontage',
 'RoofMatl',
 'HeatingQC',
 'MoSold',
 'Bsm

In [70]:
y_pred = automl.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  27387.96219002552
mse  750100472.9222673
mae  16144.498501712329
explained_variance_score 0.8929251158976506
r2 0.89292399988095


In [67]:
ser = pd.DataFrame(xgbr.feature_importances_)
ser.columns  =['feature_importance']
ser['column_name'] = x_test.columns
ser.sort_values('feature_importance',ascending=True).head(20)

<IPython.core.display.Javascript object>

,feature_importance,column_name
76,0.000000,PoolQC
13,0.000000,LowQualFinSF
39,0.000000,Street
48,0.000000,Condition2
52,0.000003,RoofMatl
73,0.000014,GarageQual
32,0.000024,MiscVal
50,0.000107,HouseStyle
56,0.000109,ExterQual
16,0.000150,BsmtHalfBath


In [68]:
ser.sort_values('feature_importance',ascending=False).head(20)

,feature_importance,column_name
2,0.391704,OverallQual
46,0.128850,Neighborhood
24,0.080949,GarageCars
59,0.064212,BsmtQual
14,0.046078,GrLivArea
68,0.038338,KitchenQual
31,0.023444,PoolArea
7,0.020478,BsmtFinSF1
70,0.020132,FireplaceQu
11,0.019833,1stFlrSF
